# Neural functional theory for inhomogeneous fluids &ndash; Tutorial

This material provides a pedagogical introduction to the concepts and methods presented in

**Neural functional theory of inhomogeneous fluids: Fundamentals and applications**  
Florian Sammüller, Sophie Hermann, Daniel de las Heras, and Matthias Schmidt (2023); [arXiv:2307.04539](https://arxiv.org/abs/2307.04539).

We show how the physics of simple fluids can be described with **many-body simulations** and **classical density functional theory** (DFT) and how the two approaches can be combined with the help of machine learning to a **neural functional theory**.
Compared to the above publication, we take a step back here and consider the one-dimensional hard rod fluid.
For this simple model, we demonstrate the central ideas of the different methods and provide hands-on code examples and exercises in the programming language Julia.
Besides the theoretical overview given here, a more in-depth account of the physical and methodological background can be found in the accompanying manuscript:

**Why neural functionals suit statistical mechanics**  
Florian Sammüller, Sophie Hermann, and Matthias Schmidt (2023).

## Part 0: Technical remarks

Running some of the tasks in this notebook requires a considerable amount of compute power, e.g. the generation of simulation data and the training of the neural network.
Keep this in mind if you are using an online service with limited resources.

Particularly for the machine learning tasks, we recommend using a GPU, as this speeds up training and inference considerably.
If you run this notebook locally and have a GPU available, open a Julia shell in a new tab, install the required packages and restart the Julia kernel of this notebook.
Further details are given in https://juliagpu.org/ where you can find the right packages for different GPU vendors.

### Setup for Google Colab

*Skip this part if you are not running this notebook in Google Colab*

In Google Colab, you can use a machine with an Nvidia GPU for free, but you first have to install Julia and the required packages.
Follow these steps each time you start a new runtime:
1. Connect to a GPU runtime:
   - In the Google Colab interface, click the arrow (&#9660;) next to *Connect* and select *Change runtime type*
   - Under *Hardware accelerator*, select some option with "GPU" in its name
   - Save the runtime settings and click *Connect*
3. Run the script below and wait a few minutes for the Julia install to finish

In [ ]:
%%shell
set -e

JULIA_VERSION="1.9.2"

if [ -z `which julia` ]; then
    JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
    echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
    BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
    URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
    wget -nv $URL -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
    
    echo "Installing IJulia package..."
    julia -e 'using Pkg; pkg"add IJulia; precompile;"' &> /dev/null
    
    echo "Installing IJulia kernel..."
    julia -e 'using IJulia; IJulia.installkernel("julia")'
    KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
    KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
    mv -f $KERNEL_NAME "$KERNEL_DIR"/julia
fi

git clone https://github.com/sfalmo/NeuralDFT-Tutorial.git
cp NeuralDFT-Tutorial/*.jl NeuralDFT-Tutorial/*.toml .

echo "Done."

3. Switch to the Julia kernel:
   - Click the arrow (&#9660;) next to *Connect* and select *Change runtime type*
   - Under *Runtime type*, select the option "julia 1.9.2" (do not change the *Hardware accelerator* which you have chosen in step 1)
5. Install the required Julia packages by running the following code cell, which will also take a few minutes

In [ ]:
import Pkg

Pkg.activate(".")
Pkg.add("CUDA")
Pkg.add("cuDNN")
Pkg.instantiate()

You are now ready to start the tutorial.
In Part 3, GPU support can simply be enabled with `using CUDA`; all required packages have just been installed.

## Part 1: Many-body simulations

Many-body simulations have long become a standard tool for the investigation of classical fluids.
Conceptually, they offer a rather straightforward way to predict the behavior of a fluid from a microscopic description, i.e. by specifying the interactions of its constituent particles.
However, simulations often come with a significant demand of computational resources.

In the following, we focus on Monte Carlo methods for the description of equilibrium systems.
Specifically, we choose the grand ensemble where the temperature $T$ and the chemical potential $\mu$ are kept fixed (we also specify the length $L$ of the one-dimensional domain).
The statistical mechanics of such a system is then determined by the equilibrium distribution function $\psi(x^{(N)}) \sim \exp(-\beta (U(x^{(N)}) - \mu N))$ where $\beta = 1 / (k_B T)$ with the Boltzmann constant $k_B$.
The potential energy $U(x^{(N)}) = u(x^{(N)}) + \sum_{i=1}^N V_\mathrm{ext}(x_i)$ of a given microstate $x^{(N)} = (x_1, x_2, \dots, x_N)$ of $N$ particles consists of a contribution due to an external potential $V_\mathrm{ext}(x)$ and of the internal energy $u(x^{(N)})$.
If the particles in the fluid possess pairwise interactions, $u(x^{(N)}) = \sum_{i=1}^N \sum_{j>i}^N \phi(|x_j - x_i|)$, where $\phi(r)$ is the interaction potential for a given distance $r$ of two particles.

Monte Carlo is based on the generation of microstates according to their known equilibrium distribution.
This is done iteratively by mutating an initial state $A$ into a new state $B$ with a probability such that a given distribution $P$ of states is kept intact.
The mutation happens in two stages: a trial transition selects a new state $B$, and a criterion $\mathrm{acc}(A \rightarrow B)$ determines if the new state $B$ shall be accepted or if the system shall be reset to the previous state $A$.
Specifically, a valid choice for this acceptance probability is the Metropolis criterion $\mathrm{acc}(A \rightarrow B) = \min(1, P(B) / P(A))$.

We now apply this scheme to the grand ensemble, which yields the standard grand canonical Monte Carlo (GCMC) method.
There are three possible trial transitions: i) a particle is moved to a new position, ii) a particle is inserted, iii) a particle is removed.
The goal distribution of states $P$ is the grand canonical equilibrium distribution $\psi$ (see above).
When considering particle displacements, the Metropolis rate becomes
$$
\mathrm{acc}(x^{(N)} \rightarrow \tilde{x}^{(N)}) = \min\left(1, \exp[-\beta(U(\tilde{x}^{(N)}) - U(x^{(N)}))]\right)
$$
for moving a particle and changing the initial microstate $x^{(N)}$ to the new configuration $\tilde{x}^{(N)}$.

Let us now illustrate this trial move with code.
For simplification, we provide some utilities for common tasks in simulation.jl, e.g. `calc_particle_interaction` for the calculation of the energy, `pbc!` for applying periodic boundary conditions, and definitions of containers (structs) which hold the state of the `System` and the `Histograms`.

In [ ]:
include("simulation.jl");

function trial_move(system::System; Δxmax=0.1)
    if isempty(system.particles)  # If no particles are in the system, do nothing
        return
    end
    i = rand(1:length(system.particles))  # Select random particle with index i
    xbefore = system.particles[i]  # Save its initial position
    Ebefore = calc_particle_interaction(system, i)  # Calculate the initial potential energy of particle i
    system.particles[i] += Δxmax * (2 * rand() - 1)  # Move particle to a new position
    pbc!(system, i)  # Apply periodic boundary conditions (this places the particle back in the box if it has moved outside of the valid range)
    Eafter = calc_particle_interaction(system, i)  # Calculate the potential energy of particle i after it has been moved
    if rand() > exp(-system.β * (Eafter - Ebefore))
        system.particles[i] = xbefore  # Trial move rejected. Reset particle to previous state
    end
end

As the particle number can fluctuate in the grand ensemble, there are additional transitions which add and remove particles at random throughout the simulation.
The acceptance probabilities of these transitions can be derived by considering particle exchanges of the system with a virtual reservoir.
We spare this derivation here and only give the results
$$
\mathrm{acc}(x^{(N)} \rightarrow x^{(N+1)}) = \min\left(1, \frac{L}{N+1} \exp[\beta(\mu - U(x^{(N+1)}) + U(x^{(N)}))]\right),
$$
$$
\mathrm{acc}(x^{(N)} \rightarrow x^{(N-1)}) = \min\left(1, \frac{N}{L} \exp[-\beta(\mu + U(x^{(N-1)}) - U(x^{(N)}))]\right).
$$

These trial transitions are already provided as `trial_insert` and `trial_delete` in simulation.jl, feel free to take a look for the implementation details.

With all transitions being specified, we can write a simulation loop which consists of an equilibration stage and a stage in which measurements take place.
For each simulation step, we perform a sweep over a fixed number of trial transitions which are chosen at random without any bias.

In [ ]:
using Dates

function sweep(system::System; transitions=10, insert_delete_probability=0.2)
    for _ in 1:transitions
        if rand() < insert_delete_probability  # Randomly select trial transition: either move or insert/delete
            rand() < 0.5 ? trial_insert(system) : trial_delete(system)  # Do trial insertions and removals with equal probability
        else
            trial_move(system)
        end
    end
end

function simulate(L::Number, μ::Number, T::Number, Vext::Function, ϕ::Function; equilibration_time=Dates.Second(1), production_time=Dates.Second(2), sweep_transitions=10)
    system = System(L, μ, T, Vext, ϕ)  # The state of the system is encapsulated in this struct
    histograms = Histograms(system)
    equilibration_start = now()
    while now() - equilibration_start < equilibration_time  # Equilibration stage, no sampling
        sweep(system; transitions=sweep_transitions)
    end
    production_start = now()
    while now() - production_start < production_time
        sweep(system; transitions=sweep_transitions)
        sample(system, histograms)
    end
    get_results(system, histograms)  # Normalizes histograms and returns (xs, ρ)
end

Equilibrium averages such as the one-body density profile $\rho(x) = \langle \sum_{i=1}^N \delta(x - x_i) \rangle$ are obtained by sampling.
For this, the particle configuration is recorded in a position-resolved histogram, which yields the desired average after normalization:

In [ ]:
function sample(system::System, histograms::Histograms)
    for x in system.particles
        bin = ceil(Int, x / L * histograms.bins)  # Calculate the bin index from the given particle position
        histograms.ρ[bin] += 1
    end
    histograms.count += 1  # Needed later for normalization
end

Let us now do some simulations of the hard rod fluid to illustrate the usage of the code.
We just pass the length $L$ of our system, the thermodynamic statepoint $\mu$ and $T$, the external potential $V_\mathrm{ext}(x)$ and the pair interaction potential $\phi(r)$ to the `simulate` function.
As a simple test case, we choose confinement between hard walls by setting $V_\mathrm{ext} = \infty$ at the boundaries of the domain.

In [ ]:
using PyPlot

L = 10.0
μ, T = 2.0, 1.0
Vext(x) = x < 0.5 || x > L - 0.5 ? Inf : 0  # Hard walls at the boundaries
ϕ(r) = r < 1.0 ? Inf : 0  # Hard core repulsion

xs, ρ = simulate(L, μ, T, Vext, ϕ; equilibration_time=Dates.Second(1), production_time=Dates.Second(2), sweep_transitions=10)

plot(xs, ρ, label="ρ"); legend()

## Part 2: Classical density functional theory

Classical DFT is founded on a minimization principle of the grand potential $\Omega[\rho]$, which can be expressed as a functional of the one-body density profile $\rho(x)$.
Its strength is therefore the reduction of a many-body problem (see Part 1) to a formally exact description on the level of one-body quantities.
By writing out entropic, external and internal contributions of the grand potential, carrying out the functional derivative $\delta \Omega[\rho] / \delta \rho(x)$ and demanding that it vanishes at the equilibrium density, one arrives at the Euler-Lagrange equation of DFT,
$$
c_1(x) = \ln\rho(x) + \beta (V_\mathrm{ext}(x) - \mu),
$$
where $c_1(x)$ is the one-body direct correlation function.
This rather abstract object attains a fundamental meaning in classical DFT as it captures the nontrivial effects of the internal interactions within the fluid on the one-body level.
Formally, it arises from a functional derivative, $c_1(x) = - \delta \beta F_\mathrm{exc}[\rho] / \delta \rho(x)$, where $F_\mathrm{exc}[\rho]$ is the excess (i.e. beyond ideal gas) free energy.
As an immediate consequence of $F_\mathrm{exc}[\rho]$ being a density functional, $c_1(x; [\rho])$ also has a functional dependence on $\rho(x)$, which is made explicit by the bracket notation.
In principle, $F_\mathrm{exc}[\rho]$ and $c_1(x; [\rho])$ also depend parametrically on the temperature $T$, but this dependence becomes trivial for the hard rod fluid.

By rearranging the above equation to
$$
\rho(x) = \exp(-\beta(V_\mathrm{ext}(x) - \mu) + c_1(x; [\rho])),
$$
we can reveal its use in actual DFT applications.
Given a suitable expression for $c_1(x; [\rho])$ (which one has to obtain somehow for a given type of model fluid), a self-consistent iteration scheme can be used to solve for $\rho(x)$.
One such method is the Picard iteration with mixing, in which the iteration is performed as follows:
$$
\rho(x) \leftarrow (1 - \alpha) \rho(x) + \alpha \rho_\mathrm{EL}(x).
$$
Here, $\alpha$ is a mixing parameter and $\rho_\mathrm{EL}(x) = \exp(-\beta(V_\mathrm{ext}(x) - \mu) + c_1(x; [\rho]))$ is the right hand side of the rearranged Euler-Lagrange equation.

A simple DFT program proceeds as follows:
The system parameters $\mu$ and $T$, an external potential $V_\mathrm{ext}(x)$ and a functional form of $c_1(x; [\rho])$ are given, and the density profile is initialized (e.g. with a constant value) on a discrete numerical grid that spans the domain of length $L$.
The iteration is then started and Picard steps are performed to update $\rho(x)$ as described above.
If the change of $\rho(x)$ between iteration steps falls below a predefined numerical tolerance, the iteration is stopped and the converged self-consistent density profile is obtained as a result.
We give an example of such a program in the following:

In [ ]:
function minimize(L::Number, μ::Number, T::Number, Vext::Function, get_c1::Function; α::Number=0.05, maxiter::Int=10000, dx::Number=0.01, floattype::Type=Float32, tol::Number=max(eps(floattype(1e3)), 1e-8))
    L, μ, T = floattype.((L, μ, T))  # Technical detail: we will use Float32 in the machine learning part as neural networks usually operate on single precision floats
    xs = collect(floattype, dx/2:dx:L)  # Construct the numerical grid
    Vext = Vext.(xs)  # Evaluate the external potential on the grid
    infiniteVext = isinf.(Vext)  # Check where Vext is infinite to set ρ = 0 there
    ρ, ρEL = zero(xs), zero(xs)  # Preallocate the density profile and an intermediate buffer for iteration
    fill!(ρ, 0.5)  # Start with a bulk density of 0.5
    c1 = get_c1(xs)  # Obtain the c1 functional for the given numerical grid
    i = 0
    while true
        ρEL .= exp.((μ .- Vext) ./ T .+ c1(ρ))  # Evaluate the RHS of the Euler-Lagrange equation
        ρ .= (1 - α) .* ρ .+ α .* ρEL  # Do a Picard iteration step to update ρ
        ρ[infiniteVext] .= 0  # Set ρ to 0 where Vext = ∞
        clamp!(ρ, 0, Inf)  # Make sure that ρ does not become negative
        Δρmax = maximum(abs.(ρ - ρEL)[.!infiniteVext])  # Calculate the remaining discrepancy to check convergence
        i += 1
        if Δρmax < tol
            println("Converged (step: $(i), ‖Δρ‖ = $(Δρmax) < $(tol) = tolerance)")
            break  # The remaining discrepancy is below the tolerance: break out of the loop and return the result
        end
        if !isfinite(Δρmax) || i >= maxiter
            println("Did not converge (step: $(i) of $(maxiter), ‖Δρ‖: $(Δρmax), tolerance: $(tol))")
            return nothing  # The iteration did not converge, there is no valid result
        end
    end
    xs, ρ
end

The way in which $c_1(x; [\rho])$ appears both in the theory as well as in the code example above seems innocuous at first.
However, the crux of DFT is finding a suitable functional expression for $c_1(x; [\rho])$ (or equivalently for $F_\mathrm{exc}[\rho]$) for a given fluid model, and much of the ongoing research deals exactly with this problem.
In Part 3, we will show how to efficiently use neural networks to capture such a nontrivial functional mapping from simulation data.

In the following, we proceed analytically as the focus lies on the hard rod system, which is the only model fluid at present where the exact excess free energy functional could be found.
This success can be traced back to the purely geometrical nature of this particular system.
For hard spheres in 3D, fundamental measure theory provides a similar geometric approach to the construction of excess functionals, although the results are no longer exact.


Returning to the exact hard rod result due to Percus, the excess free energy can be expressed as 
$$
\beta F_\mathrm{exc}[\rho] = \int \mathrm{d}x \Phi(n_0(x; [\rho]), n_1(x; [\rho]))
$$
where the free energy density has the form $\Phi(n_0, n_1) = - n_0 \ln(1 - n_1)$.
The functions $n_0(x; [\rho])$ and $n_1(x; [\rho])$ are *weighted* densities which arise from convolutions of the density profile with the weight functions $\omega_0(x) = (\delta(x-R) + \delta(x+R)) / 2$ and $\omega_1(x) = \Theta(R - |x|)$, i.e. $n_\alpha(x; [\rho]) = (\omega_\alpha \star \rho)(x)$.
From this excess free energy functional, one can easily obtain by functional differentiation the result
$$
c_1(x; [\rho]) = - \sum_{\alpha=0,1} \left(\omega_\alpha \star \frac{\partial \Phi}{\partial n_\alpha}\right)(x)
$$
for the one-body direct correlation function.

We implement a method that constructs the Percus $c_1(x; [\rho])$ for a given numerical grid in the following.
As assistance, we provide `conv_fft` to evaluate convolutions efficiently in Fourier space and `get_weights_Percus` to obtain $\omega_\alpha(x)$ on the numerical grid in the file dft.jl.

In [ ]:
include("dft.jl")

function get_c1_Percus(xs)
    ω0, ω1 = get_weights_Percus(xs)
    conv(f, g) = conv_fft(f, g; dx=xs[2]-xs[1])
    function (ρ)
        n0, n1 = conv(ω0, ρ), conv(ω1, ρ)
        ∂ϕ_∂n0 = -log.(1 .- n1)
        ∂ϕ_∂n1 = n0 ./ (1 .- n1)
        -(conv(ω0, ∂ϕ_∂n0) .+ conv(ω1, ∂ϕ_∂n1))
    end
end

Now we can perform some DFT minimizations to see if the results match the simulations.
Note that in general, you might have to adjust the mixing parameter $\alpha$ to ensure convergence of the Picard iteration (see the keyword arguments of `minimize`).

In [ ]:
using PyPlot

L = 10.0
μ, T = 2.0, 1.0
Vext(x) = x < 0.5 || x > L - 0.5 ? Inf : 0

xs, ρ = minimize(L, μ, T, Vext, get_c1_Percus)

plot(xs, ρ, label="ρ"); legend()

As we can check by comparing simulation and DFT results for the same choices of $L$, $\mu$, $T$ and $V_\mathrm{ext}(x)$, the density profiles of the hard rod fluid are consistent.
However, it is obvious that both methods have substantial (and quite contrary) restrictions:
- The simulation data is noisy, which can only be improved with longer simulation runs. This quickly becomes prohibitively expensive, in particular if one wishes to perform many individual simulations to explore the behavior for different system parameters. However, one is free to change the type of considered fluid by simply modifying the form of the internal interactions.
- The DFT calculation is fast and does not suffer from noisy results, thus enabling vast and efficient parameter studies. The results are exact for the case of hard rods, but as illustrated above, one had to find and implement a suitable density functional in order to capture intrinsic correlations. For more complex fluids, there is little hope in deriving an exact functional analytically, and the construction of good approximations often proves to be very difficult.

In the final part of this tutorial, we will combine the advantages of both methods with the help of machine learning.
We will demonstrate how to acquire an accurate and flexible representation of $c_1(x; [\rho])$ by training a neural network with simulation data.
The resulting *neural functional* can be used in the minimization scheme as seen above, but it can also reveal more fundamental information about the statistical mechanics of the considered fluid.

## Part 3: Neural functional theory

As we have seen in Part 2, classical DFT is conceptually powerful but limited by the difficulty of finding analytic expressions for $c_1(x; [\rho])$.
In the following, we will train a neural network with simulation data (see Part 1) to obtain a representation of $c_1(x; [\rho])$.

To generate a suitable data set, simulations of systems with randomized inhomogeneous external potentials $V_\mathrm{ext}(x)$ and random values of the chemical potential $\mu$ are performed.
For the construction of an appropriate form of $V_\mathrm{ext}(x)$, we combine Fourier modes, linear segments and hard walls, which are defined in the following.
We also write a method which generates a combination of these elementary functions with randomly chosen parameters.

In [ ]:
Vext_sin(x; n::Int, A::Number, φ::Number, L::Number) = A * sin(2π * x * n / L + φ)

Vext_lin(x; x1::Number, x2::Number, E1::Number, E2::Number) = x > x1 && x < x2 ? E1 + (x - x1) * (E2 - E1) / (x2 - x1) : 0

Vext_wall(x; xw::Number, L::Number) = x < xw || x > L - xw ? Inf : 0

function generate_Vext(L::Number; num_sin=4, num_lin=rand(1:5), wall=true)
    Avar = 1.0
    sin_parameters = []
    for n in 1:num_sin
        push!(sin_parameters, (n = n, A = randn() * Avar, φ = rand() * 2π, L = L))
    end
    Evar = 1.0
    lin_parameters = []
    for _ in 1:num_lin
        push!(lin_parameters, (x1 = round(rand() * L, digits=2), x2 = round(rand() * L, digits=2), E1 = randn() * Evar, E2 = randn() * Evar))
    end
    xwmax = 1.0
    wall_params = (xw = round(rand() * xwmax, digits=2), L = L)
    function (x)
        result = 0.0
        for sin_params in sin_parameters
            result += Vext_sin(x; sin_params...)
        end
        for lin_params in lin_parameters
            result += Vext_lin(x; lin_params...)
        end
        if wall
            result += Vext_wall(x; wall_params...)
        end
        result
    end
end

To get an idea of the randomized inhomogeneous environments, let us generate and plot some $V_\mathrm{ext}(x)$ profiles.
Run the following code cell a few times:

In [ ]:
using PyPlot

L = 10.0
dx = 0.01
xs = dx/2:dx:L
Vext_generated = generate_Vext(L)
plot(xs, Vext_generated.(xs), label="Vext"); legend()

We can now proceed to generate reference data by running simulations with these randomized external potentials and random values of the chemical potential as input.
Of course, this means spending some computational resources.
You can run the simulations yourself, but be aware that this does take a considerable amount of compute time (~hours) if you want to get good data.
For this, change `use_prepared_simulations` to `false` in the following code cell and set an appropriate number of simulations and the time to spend for the equilibration and production stage.
Alternatively, you could "fake" the simulations by using the exact Percus DFT as illustrated in Part 2.

For readers without the required computational resources and/or patience, we have prepared a ready-to-use data set for the following machine learning tasks, which is downloaded by default as follows:

In [ ]:
using DelimitedFiles, Downloads

use_prepared_simulations = true  # Choose if you want to use pregenerated simulations (true) or if you want to generate a simulation data set from scratch (false)

if use_prepared_simulations
    println("Downloading pregenerated simulation data set")
    
    Downloads.download("https://www.staff.uni-bayreuth.de/~bt306964/neuraldft-tutorial/data.tar", "data.tar")
    run(`tar xf data.tar`)  # You should now have a directory "data" with a bunch of simulation files
    datadir = "data"
else
    num_sim = 512
    equilibration_time = Dates.Second(10)
    production_time = Dates.Second(200)
    nthreads = Threads.nthreads()
    println("Generating simulation reference data from scratch. This will take approximately $(canonicalize(num_sim * (equilibration_time + production_time) / nthreads)).")
    
    L = 10
    ϕ(r) = r < 1.0 ? Inf : 0  # Hard core repulsion
    μmin, μmax = -7.0, 5.0
    T = 1.0
    
    datadir = mkdir("data_$(now())")
    println("Saving results to $(datadir)")
    Threads.@threads for i in 1:num_sim
        μ = μmin + rand() * (μmax - μmin)
        Vext_generated = generate_Vext(L)
        println("Simulation $(i) running...")
        xs, ρ = simulate(L, μ, T, Vext_generated, ϕ; equilibration_time, production_time)
        μloc = μ .- Vext_generated.(xs)
        writedlm("$(datadir)/$(i).dat", [xs μloc ρ])
        println("Simulation $(i) done")
    end
end

println("There are $(length(readdir(datadir))) result files in the directory $(datadir)")

If you have run your own simulations above, the results are now located in the directory `data_<timestamp>` (use the file explorer of JupyterLab on the left side).
Otherwise, proceed with the pregenerated simulations which are located in `data` after successful download and unpacking.

Let us first plot some of the data files to check if they are reasonable:

In [ ]:
using DelimitedFiles, PyPlot

datadir = "data"  # or use your own simulation results from above

sim = rand(readdir(datadir, join=true))  # Select a random simulation within the data directory
xs, μloc, ρ = eachcol(readdlm(sim))  # Read the result file and split columns
plot(xs, ρ, label="ρ"); legend()

Note that besides $\rho(x)$, we have saved the local chemical potential $\mu_\mathrm{loc}(x) = \mu - V_\mathrm{ext}(x)$ as a further one-body field.
This additional information suffices to calculate the one-body direct correlation function $c_1(x) = \ln \rho(x) - \beta \mu_\mathrm{loc}(x)$ for each $x$ where $\rho(x) \neq 0$, which is the target of our following investigation.

In Part 2, we have shown that the one-body direct correlation function cannot only be obtained pointwise by the above relation, but that it also constitutes a universal functional $c_1(x; [\rho])$ of the density profile, without having to invoke the local chemical potential $\mu_\mathrm{loc}(x)$ explicitly.
In the following, we attempt to machine-learn this functional relationship with a neural network.
For constructing the specific input-output mapping, we appeal to some physical background.
Formally, the functional dependence on $\rho(x)$ is given with respect to the profile of the whole system.
However, for short-ranged pair interactions, the influence of the surrounding density profile on the value of $c_1(x; [\rho])$ at a certain position $x$ also remains very short-ranged.
Therefore, we can adopt a *local* learning strategy: the density profile is taken only within a narrow window around a given position as input to the neural network, which is trained to yield the *scalar value* of $c_1(x; [\rho])$ at that position.
The whole one-body direct correlation profile can be recovered by evaluating the neural network at different positions across the system domain.

Therefore, prior to the actual training, the data has to be prepared accordingly, i.e. the input-output pairs of $\rho$-windows and $c_1$-values have to be generated from the full profiles.
For this, some utility functions are given in the file neural.jl.

In [ ]:
include("neural.jl")

ρ_profiles, c1_profiles = read_sim_data(datadir)
ρ_windows, c1_values = generate_inout(ρ_profiles, c1_profiles; window_bins=201)  # one central bin and 100 bins = 1σ wide to each side

size(ρ_windows), size(c1_values)

The data is now ready to be given to a neural network for training.
We choose a simple architecture with fully-connected hidden layers and continuous activation functions (e.g. `softplus`).
For the construction of the neural network and the subsequent machine learning routines, the framework [Flux.jl](https://fluxml.ai/) is used.
We also specify a standard optimizer and give the generated input-output pairs to a data loader, which automates shuffling and batching during training.
As this is a common regression task, we select the mean squared error as the loss function and the mean absolute error as the metric.
The learning rate is decreased exponentially after the first few epochs, which improves the final training result.

You can choose in the following code cell via `use_pretrained_model` whether you want to download a ready-to-use model or whether you want to do the training yourself (running the training with a GPU is recommended).
You can enable GPU support by using the appropriate package for your type of GPU (if you have one), see also https://fluxml.ai/Flux.jl/stable/gpu/ and https://juliagpu.org/.
Otherwise, the code runs as-is on the CPU (do not worry about some Info logs which state that CUDA functionality is being called).

In [ ]:
using BSON, Dates, Downloads, Flux, Printf
using CUDA  # for Nvidia GPU support (you need CUDA.jl and cuDNN.jl). For other GPU manufacturers: AMDGPU, Metal. Comment out if you do not have a GPU

use_pretrained_model = true  # Choose if you want to use the pretrained model (true) or if you want to do the training yourself (false)

if use_pretrained_model
    println("Downloading pretrained model")
    
    Downloads.download("https://www.staff.uni-bayreuth.de/~bt306964/neuraldft-tutorial/model.bson", "model.bson")
    BSON.@load "model.bson" model
else
    println("Training model from scratch")
    ρ_windows, c1_values = (ρ_windows, c1_values) |> gpu
    
    model = Chain(
        Dense(size(ρ_windows)[1] => 128, softplus),
        Dense(128 => 64, softplus),
        Dense(64 => 32, softplus),
        Dense(32 => 1)
    ) |> gpu
    
    display(model)  # Show a summary of the model with the number of fittable parameters
    
    opt = Flux.setup(Adam(), model)  # Set up a standard Adam optimizer
    
    loader = Flux.DataLoader((ρ_windows, c1_values), batchsize=256, shuffle=true)  # Initialize the DataLoader to yield shuffled batches
    
    loss(m, x, y) = Flux.mse(m(x), y)  # Use mean squared error as loss
    metric(m, x, y) = Flux.mae(m(x), y)  # Use mean absolute error as metric
    
    get_learning_rate(epoch; initial=0.0001, rate=0.03, wait=5) = epoch < wait ? initial : initial * (1 - rate)^(epoch - wait)

    model_savefile = "model_$(now()).bson"
    println("Saving model to $(model_savefile)")
    for epoch in 1:250  # Do the training loop
        learning_rate = get_learning_rate(epoch)
        Flux.adjust!(opt, learning_rate)
        @printf "Epoch: %3i (learning_rate: %.2e)..." epoch learning_rate; flush(stdout)
        Flux.train!(loss, model, loader, opt)
        @printf " loss: %.5f, metric: %.5f\n" loss(model, ρ_windows, c1_values) metric(model, ρ_windows, c1_values); flush(stdout)
        model = model |> cpu
        BSON.@save model_savefile model
        model = model |> gpu
    end
end

model

For convenience, we write a method that lets us evaluate the whole $c_1$-profile via the trained model for a given density profile.
This is necessary due to the local nature of the model, which prevents $\rho(x)$ being used directly as input.
Instead, $\rho(x)$ must be structured into appropriate windows, which are then passed to the model.

In [ ]:
function get_c1_neural(model, xs)
    window_bins = length(model.layers[1].weight[1,:])
    model = model |> gpu
    function (ρ)
        ρ_windows = generate_windows(ρ; window_bins) |> gpu  # The helper function generate_windows is defined in neural.jl
        model(ρ_windows) |> cpu |> vec  # Evaluate the model, make sure the result gets back to the CPU, and transpose it to a vector
    end
end

The central use of the neural functional is revealed when we apply it for the self-consistent calculation of density profiles.
Just like we have used the analytical functional due to Percus in Part 2, we can now employ the neural functional to obtain results for $c_1(x; [\rho])$, which are then used to update and iterate $\rho(x)$ in the minimization scheme until convergence.
Hence, this *neural* DFT is an effective means to circumvent the cumbersome search for analytical functionals.

We show in the following that neural DFT indeed yields accurate results for the considered hard rod fluid.
Exact reference data for comparison can be obtained immediately by the Percus theory (one would have to resort to simulation results for other fluid models).

In [ ]:
using PyPlot

L = 10.0
μ, T = 2.0, 1.0
Vext(x) = x < 0.5 || x > L - 0.5 ? Inf : 0

xs, ρ_neural = minimize(L, μ, T, Vext, xs -> get_c1_neural(model, xs))
xs, ρ_Percus = minimize(L, μ, T, Vext, get_c1_Percus)

plot(xs, ρ_neural, label="ρ (neural)"); plot(xs, ρ_Percus, label="ρ (Percus)", linestyle="dashed"); legend()

Although the box size has been kept constant during training and in the above example, we can immediately apply the neural functional for different choices of $L$ due to its local nature.
This also gives us the possibility to run some rather demanding quality checks.
For narrow confinement, which has clearly not been included in our training data, the hard rod fluid yields very interesting density profiles.
The following test reveals that the neural functional is able to extrapolate to these unseen cases and that the results from neural DFT match the analytical theory with surprising accuracy.

In [ ]:
using PyPlot

L = 3.0  # also try other values between 2.0 and 4.0
μ, T = 2.0, 1.0
Vext(x) = x < 0.5 || x > L - 0.5 ? Inf : 0

xs, ρ_neural = minimize(L, μ, T, Vext, xs -> get_c1_neural(model, xs))
xs, ρ_Percus = minimize(L, μ, T, Vext, get_c1_Percus)

plot(xs, ρ_neural, label="ρ (neural)"); plot(xs, ρ_Percus, label="ρ (Percus)", linestyle="dashed"); legend()

We can also increase $L$ significantly compared to the previous cases.
The ability of the neural functional to be used straightforwardly in this manner opens up the possibility of efficient multiscale investigations.
In the following, this is illustrated by considering the sedimentation of the hard rod fluid in a column with hard walls at the top and at the bottom.
The external potential is chosen to slowly increase linearly with height $x$ in order to model the effect of gravity.
Within the sedimentation column, the density closely follows the equation of state of the hard rod fluid.

In [ ]:
using PyPlot

L = 100.0  # you can try to increase L even further
μ, T = 0.0, 1.0
Vext(x) = x < 0.5 || x > L - 0.5 ? Inf : 0.01 * x  # Hard walls at the boundaries and linearly increasing within the sedimentation column

xs, ρ_neural = minimize(L, μ, T, Vext, xs -> get_c1_neural(model, xs))
xs, ρ_Percus = minimize(L, μ, T, Vext, get_c1_Percus)

plot(xs, ρ_neural, label="ρ (neural)"); plot(xs, ρ_Percus, label="ρ (Percus)", linestyle="dashed"); legend()

Besides using the trained model directly in DFT minimizations, we show in the following that one can extract much more fundamental physical information from the neural functional.
For this, we aim at implementing functional calculus on the basis of the neural network.
An important prerequisite for this is to understand the concept of automatic differentiation.
In fact, we have used autodifferentiation already above to realize the adaptation of the neural network parameters during its training.
This has been hidden within the `Flux.train!` method, which evaluates for the given training data the output of the neural network as well as gradients with respect to all trainable parameters.
The loss function then yields an error to the target output values, which can be traced back to the weights due to the availability of these gradients in order to nudge them in the "right direction", i.e. to decrease the loss.

Automatic differentiation computes these gradients by recording individual subexpressions of a given composite function, e.g. the neural network, during evaluation.
The chain rule is then applied programmatically to evaluate derivatives of the function.
As this mechanism is central to machine learning, many frameworks come with ready-to-use implementations of automatic differentiation.
Flux.jl provides the `gradient` function, which is used as follows:

In [ ]:
using Flux

f(x, y) = x^2 + sin(x * y)  # Some test function
df(x, y) = (2*x + y * cos(x * y), x * cos(x * y))  # Analytic derivative to test autodiff results

x, y = π, 2.0

println("autodiff: ", gradient(f, x, y), ", analytic: ", df(x, y))

We can now easily and efficiently derive the output of our neural functional with respect to the input density.
But what do we get out of this?

In Part 2, we have seen that the one-body direct correlation functional is obtained from the functional derivative of the excess free energy, $c_1(x; [\rho]) = -\delta \beta F_\mathrm{exc}[\rho] / \delta \rho(x)$, and that it is a central object in DFT calculations.
Applying an additional functional derivative yields another important quantity, the two-body direct correlation function 
$$
c_2(x, x'; [\rho]) = \frac{\delta c_1(x; [\rho])}{\delta \rho(x')}.
$$
In fact, a complete hierarchy of direct correlation functions $c_n(x, x', ...; [\rho])$ is defined by iterating more functional derivatives.

From fundamental physical considerations, further relations can be obtained for these direct correlation functions.
In particular, by applying Noether's theorem to statistical mechanical systems, one arrives at so-called sum rules, which constrain the interrelation of different quantities of interest.
One such sum rule which connects $c_1(x)$, $c_2(x, x')$ and $\rho(x)$ reads as follows:
$$
\nabla c_1(x) = \int \mathrm{d}x' c_2(x, x') \nabla' \rho(x').
$$
As these identities emerge from fundamental invariances of equilibrium many-body systems, they can be used as further valuable tests to check the consistency of the neural functional.

We implement in the following a method to calculate $c_2(x, x'; [\rho])$ as obtained via autodifferentiation of a given function for $c_1(x; [\rho])$.
Note that autodifferentiation can be used very generically, i.e. this gives us a way to obtain $c_2(x, x'; [\rho])$ also from the Percus $c_1(x; [\rho])$ without having to derive and implement the analytic expression ourselves.

In [ ]:
function get_c2_autodiff(c1_function, xs)
    dx = xs[2] - xs[1]
    function (ρ)
        Flux.jacobian(c1_function, ρ)[1] / dx
    end
end

First, we run some quick tests to illustrate the usage.
As test density input, we take some analytic results that follow from a randomized continuous external potential.
Run the cell a few times to see different results for $c_2(x, x'; [\rho])$.
Also try to switch out the neural correlation functional for the Percus expression, the use of autodifferentiation remains straightforward.

In [ ]:
using PyPlot

L = 10.0
μ, T = 0.0, 1.0

xs, ρ = minimize(L, μ, T, generate_Vext(L; num_sin=4, num_lin=0, wall=false), get_c1_Percus)

c1_func = get_c1_neural(model, xs)
# c1_func = get_c1_Percus(xs)  # the Percus c1 can also be used in the following autodifferentiation, no need for pen-and-paper derivation :)

c2_func = get_c2_autodiff(c1_func, xs)

c1 = c1_func(ρ)
c2 = c2_func(ρ)

imshow(c2)

Now we can check the above Noether sum rule.
The gradients are evaluated numerically via finite differences for which we have also supplied a function `finite_diff` in neural.jl.
The following plot shows the right and left hand side of the identity, which coincide when evaluating the quantities with a neural functional that has been trained sufficiently well.
Recall that we have neither given $c_2(x, x'; [\rho])$ nor the Noether identity during training.
The successful reproduction of such fundamental results implies that the neural functional does not merely interpolate values of $c_1(x, [\rho])$, but that it instead also captures the essential physics of the underlying many-body problem.

In [ ]:
dx = xs[2] - xs[1]
grad_c1 = finite_diff(c1; dx)
grad_ρ = finite_diff(ρ; dx)
rhs = c2 * grad_ρ * dx
plot(grad_c1, label="∇c1(x)"); plot(rhs, linestyle="dashed", label="∫dx' c2(x,x') ∇'ρ(x')"); legend()

To complete the functional calculus, we now consider the *inverse* of functional differentiation.
The availability of such an operation gives us direct access to the excess free energy $F_\mathrm{exc}[\rho]$, as can be deduced from the definition of $c_1(x; [\rho])$.
Formally, one obtains a useful expression for this task by considering functional line integration, which amounts to performing a line integral in function space.
By choosing a linear parameterization $\rho_a(x) = a \rho(x)$, one can derive the explicit result
$$
\beta F_\mathrm{exc}[\rho] = - \int \mathrm{d}x \rho(x) \int_0^1 \mathrm{d}a c_1(x, [\rho_a]).
$$

On the level of the neural functional (or, in fact, any representation of the one-body direct correlation functional), this expression is easy to evaluate numerically by discretization of the $a$-integral.
We write a method for this in the following.

In [ ]:
function get_Fexc_funcintegral(c1_function, xs; num_a=30)
    dx = xs[2] - xs[1]
    da = 1 / num_a
    as = da/2:da:1
    function (ρ)
        aintegral = zero(ρ)
        for a in as
            aintegral .+= c1_function(a .* ρ)
        end
        -sum(ρ .* aintegral) * dx * da
    end
end

The functional line integration is tested as above for randomized inhomogeneous environments.
We also add an explicit consistency check to the analytic Percus free energy functional.
The values of $F_\mathrm{exc}[\rho]$ as obtained by functional line integration of the neural correlation functional show almost no discrepancy to the exact analytic theory.

In [ ]:
using PyPlot

L = 10.0
μ, T = 0.0, 1.0

xs, ρ = minimize(L, μ, T, generate_Vext(L), get_c1_Percus)

c1_func = get_c1_neural(model, xs)
# c1_func = get_c1_Percus(xs)

Fexc_func = get_Fexc_funcintegral(c1_func, xs)

function get_Fexc_Percus(xs)
    dx = xs[2] - xs[1]
    ω0, ω1 = get_weights_Percus(xs)
    conv(f, g) = conv_fft(f, g; dx)
    function (ρ)
        n0, n1 = conv(ω0, ρ), conv(ω1, ρ)
        ϕ = n0 .* log.(1 .- n1)
        -sum(ϕ) * dx
    end
end

Fexc_Percus = get_Fexc_Percus(xs)

println("Fexc from analytic expression (Percus): ", Fexc_Percus(ρ))
println("Fexc from functional line integral (neural): ", Fexc_func(ρ))
plot(ρ, label="ρ"); legend()